## Importing data, preparing batch processing

In [3]:
import preprocessing as pre
import torch
import torch.nn as nn
import torch.functional as F
import torch.optim as optim
import torchtext.data as data
import copy
import all_models as am

In [ ]:
train_data, test_data, val_data, TEXT, LABEL = pre.get_data('train_small.csv', 'val_small.csv', 'test_small.csv', 100)

Connected!


/home/ubuntu/bf-repo/preprocessing.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  apply(lambda x: x[0: 1000000])


In [ ]:
TEXT.build_vocab(train_data)
LABEL.build_vocab(train_data)

In [ ]:
USE_CUDA = torch.cuda.is_available()

In [ ]:
BATCH_SIZE = 5

device = torch.device('cuda' if USE_CUDA else 'cpu')

TEXT.build_vocab(train_data)
LABEL.build_vocab(train_data)

train_it, test_it, val_it = data.BucketIterator.splits(
    (train_data, test_data, val_data), 
    batch_size = BATCH_SIZE,
    sort_key=lambda x: len(x.alj_text),
    sort_within_batch=True,
    device = device)

## Setting up model

In [ ]:
class RNN(nn.Module):
    def __init__(self, rnn_type, input_size, embedding_size, hidden_size, output_size,
                 num_layers, dropout, bidirectional, padding_idx):
        super().__init__()
        self.embedding = nn.Embedding(input_size, embedding_size, padding_idx=padding_idx)
        self.rnn = getattr(nn, rnn_type.upper())(embedding_size, hidden_size, num_layers, dropout=dropout,
                                         bidirectional=bidirectional)

        self.dropout = nn.Dropout(dropout)
        linear_inp = (hidden_size * 2 if bidirectional else hidden_size)
        self.linear = nn.Linear(hidden_size, output_size)
             
    def forward(self, input):
        embed = self.embedding(input)
        rnn_out, hidden = self.rnn(embed)
        rnn_out = rnn_out[-1]
        dropped_rnn_out = self.dropout(rnn_out)
        linear_out = self.linear(rnn_out)
        return linear_out
    
    def evaluate(self, preds, labels):
        return self.loss_fn(pred, label)


In [14]:
class WordEmbAvg(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim, pad_idx, two_layers=True, dropout_p=0.0):
        
        super().__init__()
        
        # Define an embedding layer, a couple of linear layers, and 
        # the ReLU non-linearity.

        ##YOUR CODE HERE##
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        if two_layers == True:
            self.linear1 = nn.Linear(embedding_dim, hidden_dim)
            self.linear2 = nn.Linear(hidden_dim, output_dim) 
        else:
            self.linear1 = nn.Linear(embedding_dim, output_dim)
            self.linear2 = None
        self.relu = nn.ReLU()
        self.drop_layer = nn.Dropout(p=dropout_p)

        
        
    def forward(self, text):

        ##YOUR CODE HERE##
        embedded = self.embedding(text)
        embedded = embedded.mean(0)
        if not self.linear2:
            linear1_output = self.linear1(embedded)
            output = self.relu(linear1_output)
            output = self.drop_layer(output)
            return output
        else:
            linear1_output = self.linear1(embedded)
            linear2_input = self.relu(linear1_output)
            output = self.linear2(linear2_input)
            output = self.drop_layer(output)
            return output

In [10]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_SIZE = 100
HIDDEN_SIZE = 100
OUTPUT_SIZE = 1
NUM_LAYERS = 2
DROPOUT = 0.5
BIDIRECTIONAL = False
PADDING_IDX = TEXT.vocab.stoi[TEXT.pad_token]
model = RNN('lstm', INPUT_DIM, EMBEDDING_SIZE, HIDDEN_SIZE, OUTPUT_SIZE,
         NUM_LAYERS, DROPOUT, BIDIRECTIONAL, PADDING_IDX)

LEARNING_RATE = 0.001
train_len = 0
train_pos = 0
for batch in train_it:
    train_len += len(batch.decision_binary)
    train_pos += batch.decision_binary.sum().item()
POS_WEIGHT = torch.tensor([(train_len - train_pos) / train_pos])
if USE_CUDA:
    POS_WEIGHT = POS_WEIGHT.cuda()
EPOCHS = 20
tm = am.Training_module(model, LEARNING_RATE, POS_WEIGHT, USE_CUDA, EPOCHS)

#Training the model
best_model_acc, best_model_prec, best_model_rec = tm.train_model(train_it, val_it)

Epoch 0: Dev Accuracy: 0.800000011920929 Dev Precision: nan Dev Recall: 0.0 Dev Loss:0.9057289958000183
Epoch 1: Dev Accuracy: 0.6000000238418579 Dev Precision: 0.0 Dev Recall: 0.0 Dev Loss:0.911056637763977
Epoch 2: Dev Accuracy: 0.6000000238418579 Dev Precision: 0.0 Dev Recall: 0.0 Dev Loss:0.9083573222160339
Epoch 3: Dev Accuracy: 0.800000011920929 Dev Precision: nan Dev Recall: 0.0 Dev Loss:0.8956953883171082
Epoch 4: Dev Accuracy: 0.800000011920929 Dev Precision: nan Dev Recall: 0.0 Dev Loss:0.8916605114936829
Epoch 5: Dev Accuracy: 0.6000000238418579 Dev Precision: 0.0 Dev Recall: 0.0 Dev Loss:1.6293823719024658
Epoch 6: Dev Accuracy: 0.800000011920929 Dev Precision: nan Dev Recall: 0.0 Dev Loss:0.8864431381225586
Epoch 7: Dev Accuracy: 0.800000011920929 Dev Precision: nan Dev Recall: 0.0 Dev Loss:0.8886189460754395
Epoch 8: Dev Accuracy: 0.800000011920929 Dev Precision: nan Dev Recall: 0.0 Dev Loss:0.8981332778930664
Epoch 9: Dev Accuracy: 0.800000011920929 Dev Precision: nan De

In [11]:
best_model_acc

RNN(
  (embedding): Embedding(10510, 100, padding_idx=1)
  (rnn): LSTM(100, 100, num_layers=2, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (linear): Linear(in_features=100, out_features=1, bias=True)
)

In [12]:
best_model_prec

RNN(
  (embedding): Embedding(10510, 100, padding_idx=1)
  (rnn): LSTM(100, 100, num_layers=2, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (linear): Linear(in_features=100, out_features=1, bias=True)
)

In [13]:
best_model_rec

RNN(
  (embedding): Embedding(10510, 100, padding_idx=1)
  (rnn): LSTM(100, 100, num_layers=2, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (linear): Linear(in_features=100, out_features=1, bias=True)
)

In [24]:
tm.model = best_model_acc
test_loss, test_acc, test_prec, test_rec = tm.evaluate(test_it)
print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}% | Test Prec: {test_prec*100:.2f}% | Test Rec: {test_rec*100:.2f}%')

/pytorch/aten/src/ATen/native/cudnn/RNN.cpp:1269: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters().
/pytorch/aten/src/ATen/native/cudnn/RNN.cpp:1269: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters().


Test Loss: 0.757 | Test Acc: 68.89% | Test Prec: 0.00% | Test Rec: nan%


/pytorch/aten/src/ATen/native/cudnn/RNN.cpp:1269: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters().


In [39]:
simple_nn = WordEmbAvg(INPUT_DIM, EMBEDDING_SIZE, HIDDEN_SIZE, OUTPUT_SIZE, PADDING_IDX)

In [40]:
simple_tm = am.Training_module(simple_nn, LEARNING_RATE, POS_WEIGHT, USE_CUDA, EPOCHS)

In [41]:
simple_best_model_acc, simple_best_model_prec, simple_best_model_rec = simple_tm.train_model(train_it, val_it)

Epoch 0: Dev Accuracy: 0.6000000238418579 Dev Precision: 0.0 Dev Recall: 0.0 Dev Loss:0.9329513907432556
Epoch 1: Dev Accuracy: 0.6000000238418579 Dev Precision: 0.0 Dev Recall: 0.0 Dev Loss:0.9552356600761414
Epoch 2: Dev Accuracy: 0.6000000238418579 Dev Precision: 0.0 Dev Recall: 0.0 Dev Loss:1.0157915353775024
Epoch 3: Dev Accuracy: 0.6000000238418579 Dev Precision: 0.0 Dev Recall: 0.0 Dev Loss:1.0917619466781616
Epoch 4: Dev Accuracy: 0.6000000238418579 Dev Precision: 0.0 Dev Recall: 0.0 Dev Loss:1.0736163854599
Epoch 5: Dev Accuracy: 0.6000000238418579 Dev Precision: 0.0 Dev Recall: 0.0 Dev Loss:1.0844529867172241
Epoch 6: Dev Accuracy: 0.6000000238418579 Dev Precision: 0.0 Dev Recall: 0.0 Dev Loss:1.2133562564849854
Epoch 7: Dev Accuracy: 0.6000000238418579 Dev Precision: 0.0 Dev Recall: 0.0 Dev Loss:1.2626413106918335
Epoch 8: Dev Accuracy: 0.6000000238418579 Dev Precision: 0.0 Dev Recall: 0.0 Dev Loss:1.1554826498031616
Epoch 9: Dev Accuracy: 0.6000000238418579 Dev Precision: 0

In [21]:
simple_best_model_acc

WordEmbAvg(
  (embedding): Embedding(10510, 100)
  (linear1): Linear(in_features=100, out_features=100, bias=True)
  (linear2): Linear(in_features=100, out_features=1, bias=True)
  (relu): ReLU()
  (drop_layer): Dropout(p=0.0, inplace=False)
)

In [22]:
simple_tm.model = simple_best_model_acc
simple_test_loss, simple_test_acc, simple_test_prec, simple_test_rec = simple_tm.evaluate(test_it)
print(f'Test Loss: {simple_test_loss:.3f} | Test Acc: {simple_test_acc*100:.2f}% | Test Prec: {simple_test_prec*100:.2f}% | Test Rec: {simple_test_rec*100:.2f}%')

Test Loss: 0.694 | Test Acc: 55.56% | Test Prec: 16.67% | Test Rec: nan%


### magic loop? (for vanilla NN)

* epochs
* drop-outs
* embedding
* hidden
* learning rate
* class weight?

In [25]:
import pandas as pd

In [26]:
df = pd.DataFrame(columns=['learning_rate', 'pos_weight', 'num_epochs', 'drop_outs', 'embedding', 'hidden', 'accuracy', 'precision', 'recall', 'loss', 'model_type'])

In [2]:
for batch in train_it:
    train_len += len(batch.decision_binary)
    train_pos += batch.decision_binary.sum().item()
POS_WEIGHT = torch.tensor([(train_len - train_pos) / train_pos])
if USE_CUDA:
    POS_WEIGHT = POS_WEIGHT.cuda()

for num_epoch in [5, 10, 20, 25]:
    for dropout_rate in [0, 0.1, 0.25, 0.5, 0.75]:
        for embed_dim in [20, 32, 64, 100]:
            for hidden_dim in [10, 25, 40, 50]:
                for learn_rate in [0.1, 0.01, 0.001]:
                    mod = WordEmbAvg(INPUT_DIM, embed_dim, hidden_dim, OUTPUT_SIZE, PADDING_IDX, dropout_p=dropout_rate)
                    trm = am.Training_module(mod, learn_rate, POS_WEIGHT, USE_CUDA, num_epoch)
                    best_acc, best_prec, best_rec = trm.train_model(train_it, val_it)
                    trm.model = best_acc
                    test_loss, test_acc, test_prec, test_rec = trm.evaluate(test_it)
                    row = [learn_rate, class_weight, num_epoch, dropout_rate, embed_dim, hidden_dim, test_acc*100, test_prec*100, test_rec*100, test_loss, "most_accurate"]
                    df.loc[len(df)] = row
                    trm.model = best_prec
                    test_loss, test_acc, test_prec, test_rec = trm.evaluate(test_it)
                    row = [learn_rate, class_weight, num_epoch, dropout_rate, embed_dim, hidden_dim, test_acc*100, test_prec*100, test_rec*100, test_loss, "most_precise"]
                    df.loc[len(df)] = row
                    trm.model = best_rec
                    test_loss, test_acc, test_prec, test_rec = trm.evaluate(test_it)
                    row = [learn_rate, class_weight, num_epoch, dropout_rate, embed_dim, hidden_dim, test_acc*100, test_prec*100, test_rec*100, test_loss, "most_recall"]
                    df.loc[len(df)] = row

NameError: name 'train_it' is not defined